In [3]:
# Step 1) Import packages

import pandas as pd
import numpy as np
import pathlib
import re
import json
from pathlib import Path
import hashlib
from datetime import datetime
import glob
# import pyarrow as pa
# import pyarrow.parquet as pq

In [4]:
# pip install pandas openpyxl ipykernel

## 1. Identify HMO files

### Function to detect if an Excel sheet matches the HMO preprocessed layout

In [5]:
# HELPER FUNCTIONS AND CONSTANTS
# want to skip sheets with these names...

SKIP_SHEETS = [
    "metadata", "participant characteristics", "participants", "characteristics",
    "dictionary", "data dict", "codebook", "demographics",
    "readme", "notes"
]

def _norm(s: str) -> str:
    """Normalize header tokens for robust matching."""
    s = (str(s) if s is not None else "").strip().lower()
    # normalize unit variants
    s = s.replace("μg/ml", "µg/ml").replace("ug/ml", "µg/ml")
    s = s.replace(" nmol / ml", " nmol/ml").replace("nmol /ml", "nmol/ml")
    # standardize bracket/paren forms
    s = s.replace("(", "[").replace(")", "]")
    # collapse spaces
    s = re.sub(r"\s+", " ", s)
    return s



In [6]:
# count how many 'unnamed' columns there are 
# --> later will use this to check that there are at least 50% unnamed columns for identfication of HMO sheets
def _is_unnamed(col: str) -> bool:
    return _norm(col).startswith("unnamed:")

In [7]:
# Fxn to detect whether a workbook contains an HMO-formatted sheet like the raw layout described
# Inputs: 
#      xlsx_path: path to Excel file
#      meta_rows_expected: expected number of metadata rows at top (default 6)
#      logger: optional list to collect log messages

def detect_hmo_sheet_preprocessed_layout(xlsx_path: str, meta_rows_expected: int = 6, logger: list | None = None):
    """
    Heuristically detect an HMO data sheet that matches the *raw Excel layout* you described.
    Returns: dict(is_hmo, sheet_name, reason, diagnostics)
    """
    # runs a dict with detection results + logs 
    log = logger if logger is not None else []

    # --- open workbook ---
    try:
        xl = pd.ExcelFile(xlsx_path, engine="openpyxl")
    except Exception as e:
        log.append({"level":"error","msg":"open_failed","error":str(e)})
        return {"is_hmo": False, "sheet_name": None, "reason":"open_failed", "diagnostics": log}

    # --- define wanted header patterns in the block header cues---
    wanted_headers = {
        "nmol": "hmo [nmol/ml]",
        "ug":   "hmo [µg/ml]",
        "pct":  "hmo [%]"
    }

    # iterates through all sheets in the workbook
    for sheet in xl.sheet_names:
    # lowercases the sheet name and skips obvious non-data sheets (e.g metadata) based on SKIP_SHEETS
        sname = sheet.strip().lower()
        if any(p in sname for p in SKIP_SHEETS):
            log.append({"level":"info","msg":"skip_by_name", "sheet":sheet})
            continue

        # --- read minimally to examine headers + early rows ---
        try:
            head = pd.read_excel(xl, sheet_name=sheet, header=0, nrows=12, dtype=object, engine="openpyxl")     #reads first 12 rows of the sheet with 0 as the header
        except Exception as e:
            log.append({"level":"warn","msg":"head_read_failed","sheet":sheet,"error":str(e)})
            continue

        # normalizes header strings (lowercase, strip, standardize units/brackets - see code above)
        norm_cols = [_norm(c) for c in head.columns]


        # --- count how many headers contain each block label & is true if each of the three appears at least once ---
        # A) header cues: exactly one of each group label present somewhere in row 0
        counts = {k: sum(1 for c in norm_cols if wanted_headers[k] in c) for k in wanted_headers}
        has_one_each = all(v >= 1 for v in counts.values())  # allow >=1 in case of split blocks


        # --- Portion of unnamed columns and passes if 50% are unnamed (can edit % for strictness) ---
        # B) plenty of Unnamed:_ columns (merged cells spill)
        unnamed_ratio = sum(_is_unnamed(c) for c in head.columns) / max(1, len(head.columns))
        unnamed_ok = unnamed_ratio >= 0.6  # tweak if too strict/lenient

        # --- Assuming the first 6 rows are metadata, everything after should be numeric ---
        # C) metadata vs numeric split around ~row 6
        meta_n = min(meta_rows_expected, len(head))
        after = head.iloc[meta_n:]

        # % numeric after meta (coerce)
        # Converts to numeric and measures the fraction of non-NaN cells; passes if ≥ 50% numeric.
        if not after.empty:
            numeric_ratio = after.apply(pd.to_numeric, errors="coerce").notna().mean().mean()
        else:
            numeric_ratio = 0.0
        numeric_ok = numeric_ratio >= 0.5



        # Records diagnostics for this sheet’s quick scan.
        log.append({"level":"debug","msg":"sheet_scan",
                    "sheet":sheet, "counts":counts,
                    "unnamed_ratio":float(unnamed_ratio),
                    "numeric_ratio":float(numeric_ratio)})
        
        #If the sheet fails any of the three early checks, skip to the next sheet.
        if not (has_one_each and unnamed_ok and numeric_ok):
            continue


        # --- Loads the entire sheet now (more expensive) only if the early checks passed. vValidate last-row SUM(%) ~ 100s ---
        try:
            full = pd.read_excel(xl, sheet_name=sheet, header=0, dtype=object, engine="openpyxl")
        except Exception as e:
            log.append({"level":"warn","msg":"full_read_failed","sheet":sheet,"error":str(e)})
            continue



        # --- identify percent block by last non-empty row with ~100s ---
        non_empty = full.dropna(how="all")
        if non_empty.empty:
            log.append({"level": "info", "msg": "all_empty", "sheet": sheet})
            continue
        last_row = non_empty.iloc[-1]

        # convert all cells to numeric and flag which are ~100
        vals = pd.to_numeric(last_row, errors="coerce")
        is_100 = vals.between(99.5, 100.5)

        # if no 100s at all, can't identify percent block
        if not is_100.any():
            log.append({"level": "info", "msg": "no_100s_in_last_row", "sheet": sheet})
            continue

        # find longest contiguous run of ~100 values
        idx = np.arange(len(is_100))
        true_idx = idx[is_100.to_numpy()]
        breaks = np.where(np.diff(true_idx) != 1)[0] + 1
        runs = np.split(true_idx, breaks)
        longest = max(runs, key=len)

        # compute proportion of 100s inside that run
        run_quality = is_100.to_numpy()[longest].mean()

        # require at least 90% of cells in that block to be ~100 else skip
        if run_quality < 0.9:
            log.append({
                "level": "info",
                "msg": "percent_block_quality_fail",
                "sheet": sheet,
                "run_quality": float(run_quality)
            })
            continue

        # adopt that contiguous block as the percent columns
        pct_cols = full.columns[longest]
        last_row_ok = True  # passed quality threshold


    # adds a new dictionary entry to running list of messages 
        log.append({
            "level": "debug",
            "msg": "percent_block_detected",
            "sheet": sheet,
            "n_cols": int(len(pct_cols)),
            "run_quality": float(run_quality)
        })

        # final detection trigger - only declare this as HMO if all checks passed
        # 1. has_one_each: each of the three block labels present
        # 2. unnamed_ok: at least 50% unnamed columns
        # 3. numeric_ok: at least 50% numeric after metadata rows
        # 4. last_row_ok: last row percent block quality passed

        if has_one_each and unnamed_ok and numeric_ok and last_row_ok:
            log.append({"level": "info", "msg": "hmo_detected", "sheet": sheet})
            return {
                "is_hmo": True,
                "sheet_name": sheet,
                "reason": "layout_heuristics_pass",
                "diagnostics": log
            }

    return {"is_hmo": False, "sheet_name": None, "reason": "no_match", "diagnostics": log}     #output if after all sheets checked, no matches found


### Run through Excel files in raw/ directory and print summary of detection results

In [8]:
# run through excel files in raw/ directory and print summary of detection results
# results are going to be stored in a csv file for tracking over time


def summarize_raw_detection(raw_dir: str | Path = "raw"):         #default directory is 'raw' folder (where raw data is stored)
    
    # converts str to Path object to work with dic and subfolders
    root = Path(raw_dir)

    # searches recursively for all **Excel files** in the directory and subdirectories (rglob)
    files = sorted(list(root.rglob("*.xlsx")) + list(root.rglob("*.xlsm")))

    # initalize empty list to store results
    results = []
    for f in files:
        # calls detection fxn on each file (one built above)
        res = detect_hmo_sheet_preprocessed_layout(str(f), meta_rows_expected=6, logger=[])
        results.append({
            "file": f.relative_to(root),
            "is_hmo": res["is_hmo"],
            "sheet": res.get("sheet_name") or "-",
            "reason": res.get("reason")
        })

    # if no results found, print message and stop
    if not results:
        print(f"[i] No Excel files found under: {root.resolve()}")
        return

    # create dataframe from results (tabular form)
    df = pd.DataFrame(results)

    # Sort with HMO first, HMO detections TRUE appear at top of summary
    df = df.sort_values(by="is_hmo", ascending=False)


    # Optional: use nice colors based on Boolean flag 
    df["verdict"] = df["is_hmo"].map({True: "✅ HMO", False: "❌ Not HMO"})

    print("\n📊 HMO Sheet Detection Summary\n")
    print(df[["file", "verdict", "sheet", "reason"]].to_string(index=False))


    # Save a CSV log for tracking over time — deduplicate by file
    outpath = Path("catalog") / "detection_log.csv"
    outpath.parent.mkdir(exist_ok=True)

    df.to_csv(outpath, index=False)
    print(f"\n[✓] Saved detailed results to {outpath.resolve()}")

    return df


### RUN CELL for printed output and CSV log

In [9]:
print("START")
print("cwd:", Path.cwd())
print("Folder 'raw' exists?", Path("raw").exists())

# run the summary
summarize_raw_detection("raw")
print("END")

START
cwd: /Users/kspann/Desktop/HMO Power Bi
Folder 'raw' exists? True

📊 HMO Sheet Detection Summary

                                            file   verdict    sheet                 reason
              Brooklyn/251028 Ritual_REPORT.xlsx     ✅ HMO ALL DATA layout_heuristics_pass
DHM Pooled/251028 DHM_Pooling_Monica_REPORT.xlsx     ✅ HMO ALL DATA layout_heuristics_pass
                  NeoBANK/NeoBANK AC REPORT.xlsx     ✅ HMO ALL DATA layout_heuristics_pass
 Oxford/250709 OxfordColostrum_Fadil_REPORT.xlsx     ✅ HMO ALL DATA layout_heuristics_pass
         Brooklyn/Brooklyn College Metadata.xlsx ❌ Not HMO        -               no_match
       Brooklyn/~$Brooklyn College Metadata.xlsx ❌ Not HMO        -            open_failed
            Oxford/metadata_overview_oxford.xlsx ❌ Not HMO        -               no_match

[✓] Saved detailed results to /Users/kspann/Desktop/HMO Power Bi/catalog/detection_log.csv
END


## 2. Rename Identified HMO Files

In [10]:
# ---- config (edit here, what we would like the layout to be) ----
CFG = {
    "metadata_cols": 6,
    "nmol_cols": ["2FL (nmol/mL)", "3FL (nmol/mL)", "DFLac (nmol/mL)", "3SL (nmol/mL)", "6SL (nmol/mL)",
                  "LNT (nmol/mL)", "LNnT (nmol/mL)", "LNFP I (nmol/mL)", "LNFP II (nmol/mL)", "LNFP III (nmol/mL)",
                  "LSTb (nmol/mL)", "LSTc (nmol/mL)", "DFLNT (nmol/mL)", "LNH (nmol/mL)", "DSLNT (nmol/mL)",
                  "FLNH (nmol/mL)", "DFLNH (nmol/mL)", "FDSLNH (nmol/mL)", "DSLNH (nmol/mL)", "SUM (nmol/mL)",
                  "Sia (nmol/mL)", "Fuc (nmol/mL)"],
    "ug_cols":   ["2FL (ug/mL)", "3FL (ug/mL)", "DFLac (ug/mL)", "3SL (ug/mL)", "6SL (ug/mL)",
                  "LNT (ug/mL)", "LNnT (ug/mL)", "LNFP I (ug/mL)", "LNFP II (ug/mL)", "LNFP III (ug/mL)",
                  "LSTb (ug/mL)", "LSTc (ug/mL)", "DFLNT (ug/mL)", "LNH (ug/mL)", "DSLNT (ug/mL)",
                  "FLNH (ug/mL)", "DFLNH (ug/mL)", "FDSLNH (ug/mL)", "DSLNH (ug/mL)", "SUM (ug/mL)"],
    "pct_cols":  ["2FL (%)", "3FL (%)", "DFLac (%)", "3SL (%)", "6SL (%)",
                  "LNT (%)", "LNnT (%)", "LNFP I (%)", "LNFP II (%)", "LNFP III (%)",
                  "LSTb (%)", "LSTc (%)", "DFLNT (%)", "LNH (%)", "DSLNT (%)",
                  "FLNH (%)", "DFLNH (%)", "FDSLNH (%)", "DSLNH (%)", "SUM (%)"],
    "meta_names": ["Sample#", "SampleName", "UniqueID", "Secretor", "Diversity", "Evenness"],
    "processed_log": pathlib.Path("catalog/processed_log.csv"),
}

#### Function to find what study the files are coming from

In [11]:
# --- helper: where did this file come from? (study id) ---
def infer_study_id(path: str | Path) -> str:             #can take a string like 'raw/study1/file.xlsx' or a Path object and always returns a string
    # converts to Path object so p now has p.name, p.stem, p.parts, etc.
    p = Path(path)
    # take the immediate subfolder under 'raw' if present; else fall back to file stem
    # .resolve to get absolute path parts
    # .parts gives tuple of all parts of the path (e.g '/','Users','kspann','Desktop','HMO Power Bi','raw','study1','file.xlsx')
    # if file 'raw' is in this tuple, get the position in the tuple
    try:
        parts = p.resolve().parts
        if "raw" in parts:
            raw_idx = parts.index("raw")
            # study folder = the next piece after 'raw', if it exists
            # assuming that the next folder is the study folder 
            if raw_idx + 1 < len(parts) - 1:           #safteguard to make sure we don't go out of bounds, if raw is last part, no study folder
                return parts[raw_idx + 1]
    except Exception:
        pass
    return p.stem  # fallback: filename without extension as study id


#### Function to rename files based on detection results - start with METADATA columns only
- rename first 6 columns to standard names since HMO measurement columns are messey before processing

In [14]:
# --- core: load one detected HMO sheet and rename just the metadata columns ---
### Loads and processes ONE excel file
# xlsx_path: path to Excel file
# cfg: configuration dictionary defining metadata and HMO columns
# meta_rows_expected: expected number of metadata rows at top (default 6)


def load_hmo_with_cfg(xlsx_path: str | Path, cfg: dict, meta_rows_expected: int = 6):
    """
    1) Detects the HMO sheet using your existing detector.
    2) Loads the sheet (header=0).
    3) Renames ONLY the first cfg['metadata_cols'] columns to cfg['meta_names'] (positional).
    4) Adds 'StudyID' column for provenance.
    Returns (df, info) where 'info' includes sheet name and why it was selected.
    """
    # 1) detect - uses existing detection function
    det = detect_hmo_sheet_preprocessed_layout(str(xlsx_path), meta_rows_expected=meta_rows_expected, logger=[])
    # figure out which sheet in this workbook is the HMO sheet
    # if detection fails, return None + reason
    if not det["is_hmo"]:
        return None, {"ok": False, "reason": det.get("reason", "no_match"), "diagnostics": det.get("diagnostics", [])}

    # pull out the chosen sheet name 
    sheet = det["sheet_name"]



    # 2) load selected sheet using row 0 as header, read everything as generic objects (don't coerce types yet)
    df = pd.read_excel(xlsx_path, sheet_name=sheet, header=0, dtype=object, engine="openpyxl")

    # --- FIX: if the first "data" row is textual (header-like) and the next row is numeric, drop the first row
    if len(df) >= 2:
        # what fraction of cells in a series are numeric?
        def _frac_numeric(s):
            return pd.to_numeric(s, errors="coerce").notna().mean()       #.notna() gives True for numeric, False for non-numeric, .mean() gives fraction of True values

        # apply to first row (index 0), if mostly text then row0_num = 0.00
        row0_num = _frac_numeric(df.iloc[0])   
        # apply to second row (index 1), if mostly numeric then row1_num = 0.90-1.00
        row1_num = _frac_numeric(df.iloc[1])   

        # Logic check: if first row is almost all non-numeric (<10%) AND second row is mostly numeric (>=50%), then drop first row
        if row0_num <= 0.10 and row1_num >= 0.50:
            df = df.iloc[1:].reset_index(drop=True)           #keeps all rows after index 0 and resets the row numbering to start a 0



    # 3) rename only metadata columns (positional, no assumptions about HMO headers yet)
    m = cfg["metadata_cols"]
    target_meta = cfg["meta_names"]

    # saftye check: your config must be consistent 
    if len(target_meta) != m:
        raise ValueError(f"CFG meta_names length ({len(target_meta)}) != metadata_cols ({m})")

    # copy current column names 
    new_cols = list(df.columns)
    
    # pad if the sheet somehow has fewer columns than expected (defensive)
    while len(new_cols) < m:
        new_cols.append(f"__missing_{len(new_cols)}__")

    # rename:  overwrite the first 6 column names with canonical metadata names, all HMO columns left as-is for now
    new_cols[:m] = target_meta  # force the first m names
    df.columns = new_cols

    # 4) add StudyID
    df.insert(0, "StudyID", infer_study_id(xlsx_path))

    # (optional) quick sanity note about overall column count vs. your intended total
    expected_total = cfg["metadata_cols"] + len(cfg["nmol_cols"]) + len(cfg["ug_cols"]) + len(cfg["pct_cols"])
    info = {
        "ok": True,
        "sheet": sheet,
        "reason": det.get("reason"),
        "col_count_loaded": df.shape[1],
        "col_count_expected": expected_total + 1,  # +1 for StudyID we inserted
    }
    return df, info

#### Helper fxn to apply above loader to every Excel file under /raw

In [15]:
# --- convenience: apply to everything in raw/ and print a small summary (no files written yet) ---
def stage_all_hmo_metadata(raw_dir: str | Path, cfg: dict):
    # find all Excel files under raw_dir
    root = Path(raw_dir)
    files = sorted(list(root.rglob("*.xlsx")) + list(root.rglob("*.xlsm")))
    if not files:
        print(f"[i] No Excel files under: {root.resolve()}")
        return

    print(f"[i] Staging metadata rename for {len(files)} file(s) under {root.resolve()}")
    hits = 0
    for f in files:
        df, info = load_hmo_with_cfg(f, cfg)
        if info and info.get("ok"):
            hits += 1
            print(f"  • {f.relative_to(root)}  → sheet: {info['sheet']}  (cols loaded: {info['col_count_loaded']})")
        else:
            print(f"  - {f.relative_to(root)}  (skip: {info.get('reason') if info else 'unknown'})")
    print(f"\n[✓] Done. HMO files staged (metadata renamed): {hits}/{len(files)}")

In [16]:
stage_all_hmo_metadata("raw", CFG)

[i] Staging metadata rename for 7 file(s) under /Users/kspann/Desktop/HMO Power Bi/raw
  • Brooklyn/251028 Ritual_REPORT.xlsx  → sheet: ALL DATA  (cols loaded: 69)
  - Brooklyn/Brooklyn College Metadata.xlsx  (skip: no_match)
  - Brooklyn/~$Brooklyn College Metadata.xlsx  (skip: open_failed)
  • DHM Pooled/251028 DHM_Pooling_Monica_REPORT.xlsx  → sheet: ALL DATA  (cols loaded: 69)
  • NeoBANK/NeoBANK AC REPORT.xlsx  → sheet: ALL DATA  (cols loaded: 69)
  • Oxford/250709 OxfordColostrum_Fadil_REPORT.xlsx  → sheet: ALL DATA  (cols loaded: 69)
  - Oxford/metadata_overview_oxford.xlsx  (skip: no_match)

[✓] Done. HMO files staged (metadata renamed): 4/7


In [17]:
from pathlib import Path
import pandas as pd

# load your Excel with no header so we can see the raw first row
xlsx_path = "raw/Oxford/250709 OxfordColostrum_Fadil_REPORT.xlsx"
sheet = "ALL DATA"

df_test = pd.read_excel(xlsx_path, sheet_name=sheet, header=0, nrows=2, dtype=object, engine="openpyxl")

# apply your _norm() to each header
normalized_headers = [_norm(c) for c in df_test.columns.tolist()]
print("\n--- normalized headers ---")
for h in normalized_headers[:30]:
    print(h)



--- normalized headers ---
sample name#
participant id
pp day
hmo
unnamed: 4
unnamed: 5
hmo [nmol/ml]
unnamed: 7
unnamed: 8
unnamed: 9
unnamed: 10
unnamed: 11
unnamed: 12
unnamed: 13
unnamed: 14
unnamed: 15
unnamed: 16
unnamed: 17
unnamed: 18
unnamed: 19
unnamed: 20
unnamed: 21
unnamed: 22
unnamed: 23
unnamed: 24
unnamed: 25
unnamed: 26
unnamed: 27
hmo [µg/ml]
unnamed: 29


#### Check if renamed files look correct with Example Outputs
- is opening Excel file from memory, determining HMO sheet, renaming metadata columns only inside the pandas DF and prints a summary and discards the DateFrame once it's done

In [18]:
# pick one of your confirmed HMO files
example_file = "raw/Oxford/250709 OxfordColostrum_Fadil_REPORT.xlsx"

# load it and get info back
df_one, info = load_hmo_with_cfg(example_file, CFG)

# show the first few columns and rows
print(info)
print(df_one.columns[:10])   # first 10 column names
print(df_one.head())         # first few rows to confirm renaming worked


{'ok': True, 'sheet': 'ALL DATA', 'reason': 'layout_heuristics_pass', 'col_count_loaded': 69, 'col_count_expected': 69}
Index(['StudyID', 'Sample#', 'SampleName', 'UniqueID', 'Secretor', 'Diversity',
       'Evenness', 'HMO [nmol/mL]', 'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')
  StudyID Sample# SampleName UniqueID Secretor Diversity  Evenness  \
0  Oxford     219    LAC0218    Day 4        0  5.017169  0.264062   
1  Oxford     110    LAC0221    Day 2        1  3.016376  0.158757   
2  Oxford     169    LAC0221    Day 3        1  3.438544  0.180976   
3  Oxford      80    LAC0228    Day 1        1  2.655082  0.139741   
4  Oxford     171    LAC0228    Day 3        1  3.212779  0.169094   

  HMO [nmol/mL]   Unnamed: 7   Unnamed: 8  ... Unnamed: 58 Unnamed: 59  \
0     355.56612  1462.045127     8.521367  ...    1.403627    4.678166   
1   11220.10593    847.61735   909.798048  ...    0.541853    1.711761   
2   8187.010248   543.853247   510.566294  ...    0.734474    2.405292

#### Rename columns with HMO data now
- use the position-based renaming function to rename HMO measurement columns based on their position relative to the detected headers

In [19]:
# --- core: rename HMO measurement blocks by position ---

def rename_hmo_blocks_by_position(df, cfg):
    """
    Find 'HMO [nmol/mL]', 'HMO [ug/mL]', 'HMO [%]' in df.columns (with many Unnamed),
    then replace each contiguous block (from that header up to the next block or end)
    with cfg['nmol_cols'], cfg['ug_cols'], cfg['pct_cols'] by *position*.

    Assumes the original column order within each block matches your target order.
    Keeps metadata (and StudyID) exactly as-is.
    Returns (df_renamed, audit) where audit summarizes what was renamed.
    """

    # stores the original column names and creates a normalized version for matching
    # uses helper fxn _norm defined above

    cols = list(df.columns)
    norm_cols = [_norm(c) for c in cols]


    # targets keyed by normalized block header
    # If you find "hmo [nmol/ml]" → rename that block using CFG["nmol_cols"], etc.
    targets = {
        "hmo [nmol/ml]": cfg["nmol_cols"],
        "hmo [µg/ml]":   cfg["ug_cols"],
        "hmo [%]":       cfg["pct_cols"],
    }

    # locate block starts and loop through each
    starts = {}
    for key in targets:
        try:
            starts[key] = norm_cols.index(key)        # finds the column number / index where the block starts
        except ValueError:
            starts[key] = None  # block not present set to None

    # order the blocks that actually exist by start index
    present = [(k, i) for k, i in starts.items() if i is not None]
    present.sort(key=lambda x: x[1])

    # initalize results - collect how many columns each block has and what names were assigned
    # new_cols --> makes a copy of the original columns to modify
    audit = {"blocks": []}
    new_cols = cols[:]


    # loop through each detected block and rename by position, start at block and end at the next block or end of df
    for idx, (key, start) in enumerate(present):
        end = present[idx + 1][1] if idx + 1 < len(present) else len(cols)
        block_len = end - start       #number of columns in this block
        desired = targets[key]        #desired names for this block from config

        # Build the assignment list length-matched to the actual block
        # if the block length matches desired, use desired
        # if block shorter than desired, truncate desired
        # if block longer than desired, pad with safe extra names
        if block_len == len(desired):
            assign = desired
        elif block_len < len(desired):
            # sheet has fewer cols than expected — truncate desired
            assign = desired[:block_len]
        else:
            # sheet has extra cols — pad with safe extras
            unit_tag = key.replace("hmo [", "").replace("]", "").replace("/", "_")
            extras = [f"EXTRA_{unit_tag}_{j+1}" for j in range(block_len - len(desired))]
            assign = desired + extras

        # write names into that slice
        new_cols[start:end] = assign

        audit["blocks"].append({
            "block": key, "start": start, "end": end,
            "block_len": block_len, "expected": len(desired),
            "renamed_to": assign[:3] + (["..."] if len(assign) > 6 else []) + assign[-3:]
        })

    df.columns = new_cols
    return df, audit


In [20]:
# use the loader that reads header=0 and adds StudyID + metadata rename
df_one, info = load_hmo_with_cfg(example_file, CFG)

# now rename the HMO blocks by position
df_renamed, audit = rename_hmo_blocks_by_position(df_one, CFG)

print(audit)
print(df_renamed.columns[:20])
print(df_renamed.head())


{'blocks': [{'block': 'hmo [nmol/ml]', 'start': 7, 'end': 29, 'block_len': 22, 'expected': 22, 'renamed_to': ['2FL (nmol/mL)', '3FL (nmol/mL)', 'DFLac (nmol/mL)', '...', 'SUM (nmol/mL)', 'Sia (nmol/mL)', 'Fuc (nmol/mL)']}, {'block': 'hmo [µg/ml]', 'start': 29, 'end': 49, 'block_len': 20, 'expected': 20, 'renamed_to': ['2FL (ug/mL)', '3FL (ug/mL)', 'DFLac (ug/mL)', '...', 'FDSLNH (ug/mL)', 'DSLNH (ug/mL)', 'SUM (ug/mL)']}, {'block': 'hmo [%]', 'start': 49, 'end': 69, 'block_len': 20, 'expected': 20, 'renamed_to': ['2FL (%)', '3FL (%)', 'DFLac (%)', '...', 'FDSLNH (%)', 'DSLNH (%)', 'SUM (%)']}]}
Index(['StudyID', 'Sample#', 'SampleName', 'UniqueID', 'Secretor', 'Diversity',
       'Evenness', '2FL (nmol/mL)', '3FL (nmol/mL)', 'DFLac (nmol/mL)',
       '3SL (nmol/mL)', '6SL (nmol/mL)', 'LNT (nmol/mL)', 'LNnT (nmol/mL)',
       'LNFP I (nmol/mL)', 'LNFP II (nmol/mL)', 'LNFP III (nmol/mL)',
       'LSTb (nmol/mL)', 'LSTc (nmol/mL)', 'DFLNT (nmol/mL)'],
      dtype='object')
  StudyID Sampl

### Save the Edited Excel Files

In [21]:
def file_sha256(path: str | Path) -> str:
    """Return SHA-256 hash for a file's contents."""
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

In [22]:
def process_and_stage_all(raw_dir: str | Path = "raw",
                          out_dir: str | Path = "staging",
                          cfg: dict = CFG):
    """
    Pipeline:
      - scan raw/ for Excel files
      - detect HMO sheet
      - load + rename metadata
      - rename HMO blocks by position to CFG targets
      - save cleaned data to staging/ (Parquet + CSV)
      - update catalog/processed_log.csv (dedup by file + sha256)
    """
    root = Path(raw_dir)
    out_root = Path(out_dir)
    out_root.mkdir(parents=True, exist_ok=True)

    files = sorted(list(root.rglob("*.xlsx")) + list(root.rglob("*.xlsm")))
    if not files:
        print(f"[i] No Excel files under {root.resolve()}")
        return

    # load existing manifest, if any
    manifest_path = Path("catalog") / "processed_log.csv"
    manifest_path.parent.mkdir(exist_ok=True)
    if manifest_path.exists():
        manifest = pd.read_csv(manifest_path)
    else:
        manifest = pd.DataFrame(columns=[
            "file", "sheet", "sha256", "rows", "cols",
            "staged_parquet", "staged_csv", "processed_at"
        ])

    new_rows = []

    hits = 0
    for f in files:
        rel = f.relative_to(root)
        try:
            df0, info = load_hmo_with_cfg(f, cfg)
            if not (info and info.get("ok")):
                print(f"  - {rel}  (skip: {info.get('reason') if info else 'unknown'})")
                continue

            # rename HMO blocks by position
            df1, audit = rename_hmo_blocks_by_position(df0, cfg)

            # build output paths (mirror raw/ structure)
            out_parquet = out_root / rel.with_suffix(".parquet")
            out_csv = out_root / rel.with_suffix(".csv")
            out_parquet.parent.mkdir(parents=True, exist_ok=True)

            # save
            try:
                df1.to_parquet(out_parquet, index=False)
            except Exception:
                # parquet requires pyarrow/fastparquet; CSV is the fallback
                pass
            df1.to_csv(out_csv, index=False)

            # compute file hash for dedup
            sha = file_sha256(f)

            new_rows.append({
                "file": str(rel),
                "sheet": info.get("sheet"),
                "sha256": sha,
                "rows": int(df1.shape[0]),
                "cols": int(df1.shape[1]),
                "staged_parquet": str(out_parquet) if out_parquet.exists() else "",
                "staged_csv": str(out_csv),
                "processed_at": datetime.utcnow().isoformat(timespec="seconds") + "Z",
            })

            hits += 1
            print(f"  • {rel}  → sheet: {info['sheet']}  → saved to {out_csv.name}")

        except Exception as e:
            print(f"  ! {rel}  (error: {e})")

    # update manifest (dedup by file+sha256, keep latest)
    if new_rows:
        upd = pd.DataFrame(new_rows)
        manifest = (
            pd.concat([manifest, upd], ignore_index=True)
              .drop_duplicates(subset=["file", "sha256"], keep="last")
              .sort_values(["file", "processed_at"])
        )
        manifest.to_csv(manifest_path, index=False)

    print(f"\n[✓] Staged {hits}/{len(files)} file(s). Manifest: {manifest_path.resolve()}")


In [23]:
process_and_stage_all("raw", "staging", CFG)

/var/folders/c8/_21cxjf54_5d0qsx2dqt5qgc0000gp/T/ipykernel_7374/1667141852.py:71: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": datetime.utcnow().isoformat(timespec="seconds") + "Z",


  • Brooklyn/251028 Ritual_REPORT.xlsx  → sheet: ALL DATA  → saved to 251028 Ritual_REPORT.csv
  - Brooklyn/Brooklyn College Metadata.xlsx  (skip: no_match)
  - Brooklyn/~$Brooklyn College Metadata.xlsx  (skip: open_failed)


/var/folders/c8/_21cxjf54_5d0qsx2dqt5qgc0000gp/T/ipykernel_7374/1667141852.py:71: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": datetime.utcnow().isoformat(timespec="seconds") + "Z",


  • DHM Pooled/251028 DHM_Pooling_Monica_REPORT.xlsx  → sheet: ALL DATA  → saved to 251028 DHM_Pooling_Monica_REPORT.csv


/var/folders/c8/_21cxjf54_5d0qsx2dqt5qgc0000gp/T/ipykernel_7374/1667141852.py:71: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": datetime.utcnow().isoformat(timespec="seconds") + "Z",


  • NeoBANK/NeoBANK AC REPORT.xlsx  → sheet: ALL DATA  → saved to NeoBANK AC REPORT.csv
  • Oxford/250709 OxfordColostrum_Fadil_REPORT.xlsx  → sheet: ALL DATA  → saved to 250709 OxfordColostrum_Fadil_REPORT.csv
  - Oxford/metadata_overview_oxford.xlsx  (skip: no_match)

[✓] Staged 4/7 file(s). Manifest: /Users/kspann/Desktop/HMO Power Bi/catalog/processed_log.csv


/var/folders/c8/_21cxjf54_5d0qsx2dqt5qgc0000gp/T/ipykernel_7374/1667141852.py:71: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": datetime.utcnow().isoformat(timespec="seconds") + "Z",


### QC - check that all files in staging/ have expected columns

In [24]:
df = pd.read_csv("staging/Oxford/250709 OxfordColostrum_Fadil_REPORT.csv")
df.head()

,StudyID,Sample#,SampleName,UniqueID,Secretor,Diversity,Evenness,2FL (nmol/mL),3FL (nmol/mL),DFLac (nmol/mL),...,LSTb (%),LSTc (%),DFLNT (%),LNH (%),DSLNT (%),FLNH (%),DFLNH (%),FDSLNH (%),DSLNH (%),SUM (%)
0,Oxford,219.0,LAC0218,Day 4,0.0,5.017169,0.264062,355.566120,1462.045127,8.521367,...,1.403627,4.678166,2.256882,1.151817,2.509531,1.511975,3.154562,0.896466,1.027294,100.0
1,Oxford,110.0,LAC0221,Day 2,1.0,3.016376,0.158757,11220.105930,847.617350,909.798048,...,0.541853,1.711761,4.616942,0.407322,1.484937,0.334749,0.512270,0.456914,1.008464,100.0
2,Oxford,169.0,LAC0221,Day 3,1.0,3.438544,0.180976,8187.010248,543.853247,510.566294,...,0.734474,2.405292,5.432946,0.480734,1.318225,0.716340,0.410192,0.409398,0.785703,100.0
3,Oxford,80.0,LAC0228,Day 1,1.0,2.655082,0.139741,11916.649738,1332.573790,3148.458365,...,0.316692,2.366759,2.589544,0.372483,0.718322,0.282532,0.374279,0.402692,0.726981,100.0
4,Oxford,171.0,LAC0228,Day 3,1.0,3.212779,0.169094,8743.543742,384.033069,274.869200,...,0.435816,2.695814,3.396190,0.843448,1.346223,1.373897,0.201152,0.386176,0.486536,100.0


In [25]:
## NEED TO FIX
### extra headers + need to have N/A fill in as N/A not 0

df2 = pd.read_csv("staging/NeoBANK/NeoBANK AC REPORT.csv")
df2.head()

,StudyID,Sample#,SampleName,UniqueID,Secretor,Diversity,Evenness,2FL (nmol/mL),3FL (nmol/mL),DFLac (nmol/mL),...,LSTb (%),LSTc (%),DFLNT (%),LNH (%),DSLNT (%),FLNH (%),DFLNH (%),FDSLNH (%),DSLNH (%),SUM (%)
0,NeoBANK,NaN,NB00003_M_27,NaN,0.0,11.488684,0.604668,18.447513,0.0,10.982119,...,0.0,6.791851,2.988888,0.0,4.845879,0.0,5.489707,3.491142,8.852420,100.0
1,NeoBANK,NaN,NB00003_M_26,NaN,0.0,13.219119,0.695743,43.031286,0.0,27.078496,...,0.0,7.147526,5.955162,0.0,5.859766,0.0,3.450584,5.945602,3.478054,100.0
2,NeoBANK,NaN,NB00010_M_4,NaN,1.0,5.657253,0.297750,3395.350216,0.0,223.783707,...,0.0,2.154859,8.867287,0.0,3.186635,0.0,0.702712,1.267584,1.725427,100.0
3,NeoBANK,NaN,NB00004_M_4,NaN,1.0,3.217685,0.169352,4806.632162,0.0,16.587343,...,0.0,2.961345,0.723994,0.0,1.725373,0.0,0.625764,0.357418,0.757477,100.0
4,NeoBANK,NaN,NB00014_M_1,NaN,1.0,4.052705,0.213300,7015.074557,0.0,1057.119594,...,0.0,4.505099,5.433000,0.0,1.786407,0.0,0.261725,0.329526,1.013101,100.0


In [26]:
df3 = pd.read_csv("staging/Brooklyn/251028 Ritual_REPORT.csv")
df3.head(10)

,StudyID,Sample#,SampleName,UniqueID,Secretor,Diversity,Evenness,2FL (nmol/mL),3FL (nmol/mL),DFLac (nmol/mL),...,LSTb (%),LSTc (%),DFLNT (%),LNH (%),DSLNT (%),FLNH (%),DFLNH (%),FDSLNH (%),DSLNH (%),SUM (%)
0,Brooklyn,NaN,1_wk0,NaN,1.0,4.243732,0.223354,4823.760301,907.884125,431.701898,...,0.657582,1.539055,7.727516,1.062247,0.878797,1.525802,0.501036,0.277526,1.168600,100.0
1,Brooklyn,NaN,2_wk0,NaN,1.0,4.227676,0.222509,4987.623510,394.956418,230.992026,...,0.775196,2.294154,5.837840,0.438142,1.043658,0.859975,0.234599,0.134101,0.801378,100.0
2,Brooklyn,NaN,3_wk0,NaN,0.0,6.042060,0.318003,21.248877,2003.290759,26.294217,...,1.916969,2.107217,3.457632,1.357757,2.646295,2.872774,0.721898,1.394920,1.714236,100.0
3,Brooklyn,NaN,4_wk0,NaN,1.0,3.976469,0.209288,6436.767823,1322.437254,789.093272,...,0.335027,2.553792,7.345609,0.810354,0.600058,1.566644,0.708391,0.738059,2.326260,100.0
4,Brooklyn,NaN,5_wk0,NaN,1.0,6.611335,0.347965,1827.468986,2458.192657,474.061703,...,0.742212,1.319510,7.452187,1.792702,1.109865,1.676793,0.553360,1.593470,1.236606,100.0
5,Brooklyn,NaN,6_wk0,NaN,1.0,5.955614,0.313453,2865.986873,1302.889945,276.554434,...,0.595636,0.729618,4.972580,1.256449,0.717185,4.510514,2.528188,1.979988,1.736977,100.0
6,Brooklyn,NaN,7_wk0,NaN,1.0,3.494044,0.183897,4830.926822,1140.824006,400.920458,...,0.390690,1.673088,4.772315,0.977019,0.794612,1.636664,0.813942,0.508519,1.380320,100.0
7,Brooklyn,NaN,8_wk0,NaN,1.0,6.913353,0.363861,1963.699902,392.644623,99.219019,...,1.178596,1.923022,0.052532,1.213210,1.917010,6.137282,0.994703,0.550909,3.415456,100.0
8,Brooklyn,NaN,9_wk0,NaN,1.0,3.611481,0.190078,5928.021144,819.649569,618.875148,...,0.732335,0.976494,6.547450,0.816195,1.213712,1.383893,0.411465,0.242175,0.797096,100.0
9,Brooklyn,NaN,10_wk0,NaN,1.0,6.314258,0.332329,2569.111929,171.317492,72.477156,...,1.503221,2.587682,0.133634,2.582821,2.656251,3.985244,0.659404,0.287181,2.279274,100.0


### Save a copy of the renamed HMO files with a hash and timestamp

In [27]:
# define function with two folders, where the files live and where to write the merged output
# turn strings (text inside quotes) into Path objects for easier path manipulation
def merge_staging_csvs(staging_dir="staging", out_dir="staging/_merged"):
    staging = Path(staging_dir)
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    # find all CSVs under staging/ recursively
    #skip anything already inside _merged/
    files = [p for p in staging.rglob("*.csv") if out not in p.parents]
    
    #if no files found, print message and return empty dataframe
    if not files:
        print(f"[i] No CSVs under {staging.resolve()}")
        return pd.DataFrame()

    # load each CSV and add a __source_file column to show where each row came from
    # stash in list
    frames = []
    for f in sorted(files):
        df = pd.read_csv(f)
        df["__source_file"] = str(f.relative_to(staging))
        frames.append(df)

    # glue all DFs together and reset row index
    merged = pd.concat(frames, ignore_index=True)

    # Optional: drop rows that are completely empty across all columns
    merged = merged.dropna(how="all")

    # Optional: simple dedup (keep last), removes exact duplicate rows
    # merged = merged.drop_duplicates()

    out_csv = out / "hmo_merged.csv"
    merged.to_csv(out_csv, index=False)
    print(f"[✓] Merged {len(files)} file(s) → {out_csv} ({merged.shape[0]} rows, {merged.shape[1]} cols)")
    return merged

# Run it:
merged_df = merge_staging_csvs()


[✓] Merged 4 file(s) → staging/_merged/hmo_merged.csv (1200 rows, 70 cols)
